# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: LoRA, compatible with model and suitable for smaller datasets
* Model: GPT-2, lightweight and compatible with LoRA
* Evaluation approach: evaluate with hugging face trainer
* Fine-tuning dataset: IMDb (movie review dataset)

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
!pip install -U datasets scikit-learn peft evaluate accelerate bitsandbytes transformers==4.30.0
!pip install --upgrade transformers
!pip install --upgrade peft

Defaulting to user installation because normal site-packages is not writeable
  Using cached transformers-4.30.0-py3-none-any.whl (7.2 MB)
  Using cached tokenizers-0.13.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.0
    Uninstalling transformers-4.47.0:
      Successfully uninstalled transformers-4.47.0
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
auto-gptq 0.4.2 requires transformers>=4.31.0, but you have transformers 4.30.0 which is incompatible.
Defaulting to us

  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.30.0
    Uninstalling transformers-4.30.0:
      Successfully uninstalled transformers-4.30.0
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
trl 0.12.2 requires transformers<4.47.0, but you have transformers 4.47.0 which is incompatible.
Defaulting to user installation because normal site-packages is not writeable


LOAD DATASET

In [2]:
# Load dataset
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("ucirvine/sms_spam")


# Split the dataset into train and test datasets
dataset = dataset["train"].train_test_split(test_size=0.2, seed=42)
train_dataset = dataset["train"]
test_dataset = dataset["test"]
print(f"Training size: {len(train_dataset)}, Testing size: {len(test_dataset)}")

Training size: 4459, Testing size: 1115


In [3]:
id2label = {0: "ham", 1: "spam"}
label2id = {"ham": 0, "spam": 1}

In [4]:
#not spam
print(f"""
label: {id2label[train_dataset[0]['label']]}
sms:
{train_dataset[0]['sms']}
""")


label: ham
sms:
K..then come wenever u lik to come and also tel vikky to come by getting free time..:-)




In [5]:
#spam
print(f"""
label: {id2label[train_dataset[4]['label']]}
sms:
{train_dataset[4]['sms']}
""")


label: spam
sms:
U can WIN £100 of Music Gift Vouchers every week starting NOW Txt the word DRAW to 87066 TsCs www.ldew.com SkillGame,1Winaweek, age16.150ppermessSubscription




IMPORT LIBRARIES

In [6]:
import torch
import transformers
import numpy as np
from transformers import (
    AutoModelForSequenceClassification, TrainingArguments, BitsAndBytesConfig,
    DataCollatorWithPadding, Trainer, pipeline
)
from transformers.pipelines.pt_utils import KeyDataset
from peft import LoraConfig, AutoPeftModelForSequenceClassification, TaskType, get_peft_model

/opt/conda/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https:/

In [7]:
from peft import LoraConfig, AutoPeftModelForCausalLM
import torch
import transformers
from trl import SFTTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig, pipeline
from transformers.pipelines.pt_utils import KeyDataset

LOAD MODEL

In [8]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("mrm8488/bert-tiny-finetuned-sms-spam-detection")
model = AutoModelForSequenceClassification.from_pretrained("mrm8488/bert-tiny-finetuned-sms-spam-detection")

TOKENIZATION

In [9]:
# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['sms'], padding="max_length", truncation=True, max_length=512)

# Tokenizing train and test datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Set the format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Check the tokenized format
print(f"Tokenized example from train dataset: {train_dataset[0]}")


Tokenized example from train dataset: {'label': tensor(0), 'input_ids': tensor([  101,  1047,  1012,  1012,  2059,  2272, 19181, 22507,  1057,  5622,
         2243,  2000,  2272,  1998,  2036, 10093,  6819, 19658,  2100,  2000,
         2272,  2011,  2893,  2489,  2051,  1012,  1012,  1024,  1011,  1007,
          102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,

EVALUATE DATA ON TEST DATASET

In [10]:
# Evaluate the model on the test dataset without fine-tuning
from sklearn.metrics import accuracy_score

# Load model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained("mrm8488/bert-tiny-finetuned-sms-spam-detection")
tokenizer = AutoTokenizer.from_pretrained("mrm8488/bert-tiny-finetuned-sms-spam-detection")

# Define a Trainer to evaluate the model
trainer = Trainer(
    model=model,                         # Pre-trained model
    args=TrainingArguments(
        output_dir="./results",           # output directory
        num_train_epochs=2,               # set the number of epochs to 2
        per_device_train_batch_size=16,   # batch size for training
        per_device_eval_batch_size=16,    # batch size for evaluation
        logging_dir="./logs",             # directory for storing logs
    ),
    train_dataset=train_dataset,          # Pass the train dataset
    eval_dataset=test_dataset,            # Pass the eval dataset
    compute_metrics=lambda p: {"accuracy": accuracy_score(p.predictions.argmax(axis=-1), p.label_ids)}  # compute accuracy
)

# Evaluate the model
eval_results = trainer.evaluate()

# Print evaluation results
print(f"Evaluation results before fine-tuning: {eval_results}")

Evaluation results before fine-tuning: {'eval_loss': 0.1065613329410553, 'eval_model_preparation_time': 0.0012, 'eval_accuracy': 0.9847533632286996, 'eval_runtime': 1.407, 'eval_samples_per_second': 792.488, 'eval_steps_per_second': 49.753}


Finetune the model

In [11]:
# Fine-tuning the model
trainer.train()

# Save the model after fine-tuning
trainer.save_model("./fine_tuned_model")

Step,Training Loss
500,0.062700


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

CREATING PEFT CONFIG

In [12]:
from peft import LoraConfig, get_peft_model
from datasets import Dataset

# Define LoRA config
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    task_type="CAUSAL_LM"
)

# Apply LoRA to your model
peft_model = get_peft_model(model, lora_config)

# Tokenize the datasets (train and test)
def tokenize_function(examples):
    return tokenizer(examples["sms"], padding="max_length", truncation=True)

# Apply tokenization
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Set the format for PyTorch (input_ids, attention_mask, labels)
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# Training configuration
training_config = {
    "do_eval": False,
    "learning_rate": 2e-4,
    "num_train_epochs": 2,
    "max_steps": -1,
    "output_dir": "./checkpoint_dir",
    "overwrite_output_dir": True,
    "per_device_train_batch_size": 4,
    "gradient_accumulation_steps": 2,
    "remove_unused_columns": True,
    "save_steps": 100,
    "save_total_limit": 1,
    "seed": 0,
    "warmup_ratio": 0.2,
}

# Convert to TrainingArguments
train_conf = TrainingArguments(**training_config)

# Trainer
trainer = Trainer(
    model=peft_model,
    args=train_conf,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
)

# Start the training
train_result = trainer.train()

# Save the PEFT model
trainer.save_model("./fine_tuned_peft_model")

/tmp/ipykernel_1569/184066410.py:48: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,0.040400
1000,0.033800


In [13]:
# Delete the model and trainer and free up GPU memory
del model
del trainer
torch.cuda.empty_cache()

In [14]:
from peft import PeftModelForSequenceClassification

# Reload the base model first
from transformers import AutoModelForSequenceClassification

base_model = AutoModelForSequenceClassification.from_pretrained(
    "mrm8488/bert-tiny-finetuned-sms-spam-detection",
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    device_map=0,
)

# Load the PEFT adapter and merge it with the base model
model = PeftModelForSequenceClassification.from_pretrained(
    base_model,
    training_config['output_dir'],  # Path where the LoRA adapter was saved
)

# merge LoRA weights into the base model
merged_model = model.merge_and_unload()

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

LOAD SAVED PEFT MODEL WEIGHTS

In [15]:
import evaluate
from transformers import Trainer, TrainingArguments

# Load the evaluation metric (accuracy)
metric = evaluate.load("accuracy")

# Define the compute_metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    accuracy = metric.compute(predictions=predictions, references=labels)
    return accuracy

# Reload the tokenizer
tokenizer = AutoTokenizer.from_pretrained("mrm8488/bert-tiny-finetuned-sms-spam-detection")

# Set up the test dataset for evaluation
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# Reload training arguments for evaluation
eval_args = TrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=16,
    do_eval=True,
    logging_dir="./logs",
    report_to="none",
)

# Initialize the Trainer for evaluation
trainer = Trainer(
    model=merged_model,
    args=eval_args,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Evaluate the model on the test dataset
eval_results = trainer.evaluate(eval_dataset=test_dataset)

# Print evaluation results
print("Evaluation Results after PEFT fine-tuning:")
print(eval_results)

/tmp/ipykernel_1569/707995959.py:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Evaluation Results after PEFT fine-tuning:
{'eval_loss': 0.06577115505933762, 'eval_model_preparation_time': 0.0012, 'eval_accuracy': 0.9847533632286996, 'eval_runtime': 0.6127, 'eval_samples_per_second': 1819.859, 'eval_steps_per_second': 114.251}


In [16]:
# Evaluate the foundation (base) model
trainer_base = Trainer(
    model=base_model,  # This is the foundation model without PEFT
    args=eval_args,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Evaluate the base model
base_results = trainer_base.evaluate(eval_dataset=test_dataset)
print("Evaluation Results for Foundation Model (Base):")
print(base_results)

# Evaluate the fine-tuned PEFT model (this part assumes you've already run the evaluation above)
peft_results = eval_results  # Results from the PEFT fine-tuned model

# Compare the results
print("\nComparison of Foundation vs Fine-Tuned PEFT Model:")
print(f"Base Model Accuracy: {base_results['eval_accuracy']:.4f}")
print(f"PEFT Model Accuracy: {peft_results['eval_accuracy']:.4f}")
accuracy_improvement = peft_results['eval_accuracy'] - base_results['eval_accuracy']
print(f"Accuracy Improvement: {accuracy_improvement:.4f}")

# (Optional) Compare other metrics like loss
print(f"\nBase Model Loss: {base_results['eval_loss']:.4f}")
print(f"PEFT Model Loss: {peft_results['eval_loss']:.4f}")
loss_reduction = base_results['eval_loss'] - peft_results['eval_loss']
print(f"Loss Reduction: {loss_reduction:.4f}")


/tmp/ipykernel_1569/2932836959.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_base = Trainer(


Evaluation Results for Foundation Model (Base):
{'eval_loss': 0.06577115505933762, 'eval_model_preparation_time': 0.0011, 'eval_accuracy': 0.9847533632286996, 'eval_runtime': 0.5697, 'eval_samples_per_second': 1957.342, 'eval_steps_per_second': 122.882}

Comparison of Foundation vs Fine-Tuned PEFT Model:
Base Model Accuracy: 0.9848
PEFT Model Accuracy: 0.9848
Accuracy Improvement: 0.0000

Base Model Loss: 0.0658
PEFT Model Loss: 0.0658
Loss Reduction: 0.0000
